This notebook will be collected automatically at **6pm on Monday** from `/home/data_scientist/assignments/Week13` directory on the course JupyterHub server. If you work on this assignment on the course Jupyterhub server, just make sure that you save your work and instructors will pull your notebooks automatically after the deadline. If you work on this assignment locally, the only way to submit assignments is via Jupyterhub, and you have to place the notebook file in the correct directory with the correct file name before the deadline.

1. Make sure everything runs as expected. First, restart the kernel (in the menubar, select `Kernel` → `Restart`) and then run all cells (in the menubar, select `Cell` → `Run All`).
2. Make sure you fill in any place that says `YOUR CODE HERE`. Do not write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed by the autograder.
3. Do not change the file path or the file name of this notebook.
4. Make sure that you save your work (in the menubar, select `File` → `Save and CheckPoint`)

# Problem 13.2. Cassandra

In this problem, we use the Cassandra Python database driver to execute CQL (Cassandra Query Language) queries.

In [ ]:
import os
import json
import cassandra
from cassandra.cluster import Cluster
from cassandra.policies import WhiteListRoundRobinPolicy
from cassandra.query import dict_factory
from cassandra.cqlengine import connection, management

from nose.tools import assert_equal, assert_true, assert_is_instance

We use the historical weather data from [Weather Underground](http://www.wunderground.com/) to create a database. For more information on this data set, see [Week 4 Problem 1](https://github.com/UI-DataScience/info490-sp16/blob/master/Week4/assignments/w4p1.ipynb).

In [ ]:
fpath = '/home/data_scientist/data/weather'
fname = 'weather_kord_2001_0101.json'

with open(os.path.join(fpath, fname)) as f:
    weather_json = json.load(f)

For simplicity, we use only two attributes, `conds` and `visi`.

In [ ]:
observations = weather_json['history']['observations']
observations = [{key: value for key, value in obs.items() if key in ['conds', 'visi']} for obs in observations]

print('There are {} dictionaries in the list.'.format(len(observations)))
print('The first element is {}'.format(observations[0]))

We use the course Cassandra server, hosted by Micrsoft Azure, on 40.124.12.119 on the default port of 9042:

In [ ]:
cassandra_ips = ['40.124.12.119']

# Establish a connection to Cassandra

# The Policy is necessary to allow Cassandra to run on Azure.
pcy = WhiteListRoundRobinPolicy(cassandra_ips)

# Create Connection
cluster = Cluster(contact_points=cassandra_ips, load_balancing_policy=pcy)
session = cluster.connect()

print('Cluster Name: {0}'.format(cluster.metadata.cluster_name))
for host in cluster.metadata.all_hosts():
    print('{0}: Host: {1} in {2}'\
          .format(host.datacenter, host.address, host.rack))

To provide distinct environments for each student, each student will create their own keyspace in the shared Cassandra cluster by using their netids.

In [ ]:
# Filename containing user's netid
fname = '/home/data_scientist/users.txt'
with open(fname, 'r') as fin:
    netid = fin.readline().rstrip()

# We will delete our working directory if it exists before recreating.
ks_name = '{0}'.format(netid)

session.row_factory = dict_factory

connection.set_session(session)

# Explicitly set session hosts, this removes annoying warnings.
connection.session.hosts = cassandra_ips

# Drop Keyspace if it exists
if ks_name in cluster.metadata.keyspaces:
    management.drop_keyspace(ks_name)

# Create Keyspace
management.create_keyspace_simple(ks_name, 1)

# Set keyspace for this session
# Note: If keyspace exists in Cassandra instance, this is only line we need.
session.set_keyspace(ks_name)

# Display all non-system keyspaces.
# Do not change to a different keyspace!

keys = [val for val in sorted(cluster.metadata.keyspaces.keys()) if 'system' not in val]
for ks in keys:
    print(ks)

print('\nCQL Query to recreate this keyspace:')
print(40*'-')
print(cluster.metadata.keyspaces[ks_name].export_as_string())

We first drop the table if it exists to ensure a clean slate before we create our new schema and insert data. Note that the table name we will be using in this problem is `weather`.

In [ ]:
def drop_table(session):
    '''
    Drops "weather" table if exists.
    
    Parameters
    ----------
    session: A cassandra.cluster.Session instance.
    
    Returns
    -------
    A cassandra.cluster.ResultSet instance.
    '''
    result = session.execute('DROP TABLE IF EXISTS weather;')
    return result

result = drop_table(session)

assert_is_instance(result, cassandra.cluster.ResultSet)
assert_equal(result.column_names, None)
assert_equal(len(result.current_rows), 0)

## Creating Table

- Craete a `weather` table that has the following 4 columns:
  - `id` (`INT`)
  - `date` (`TEXT`)
  - `conds` (`TEXT`)
  - `visi` (`FLOAT`)
- We will use the `WHERE` clause on both `conds` and `id` later in this problem, so create an appropriate primary key.

In [ ]:
def create_table(session):
    '''
    Craetes a "weather" table with four attributes:
    id, date, conds, and visi.
    
    Parameters
    ----------
    session: A cassandra.cluster.Session instance.
    
    Returns
    -------
    A cassandra.cluster.ResultSet instance.
    '''
    
    # YOUR CODE HERE
    
    return result

In [ ]:
create_result = create_table(session)
print(create_result.response_future)

In [ ]:
result = session.execute('SELECT * FROM weather;')
assert_is_instance(result, cassandra.cluster.ResultSet)
assert_equal(set(result.column_names), {'date', 'id', 'conds', 'visi'})
assert_equal(len(result.current_rows), 0)

## Inserting Data

- Add the weather data `observations` to our Cassandra database.
- The `date` column should be `"0101"` for all rows.
- The `id` column should start from 1, and `id == 1` should correspond to the first element of `data`, `id == 2` to the second element, and so on.

In [ ]:
def insert_data(session, data):
    '''
    Adds new rows to Cassandra database.
    
    Parameters
    ----------
    session: A cassandra.cluster.Session instance.
    data: A list of dictionaries.
    
    Returns
    -------
    A cassandra.cluster.ResultSet instance.
    '''
    
    # YOUR CODE HERE
    
    return result

In [ ]:
insert_result = insert_data(session, observations)
print(insert_result.response_future)

In [ ]:
result = session.execute('SELECT * FROM weather;')
assert_is_instance(result, cassandra.cluster.ResultSet)
assert_equal(len(result.current_rows), len(observations))
assert_equal(
    {row['conds'] for row in result.current_rows},
    {obs['conds'] for obs in observations}
    )
assert_equal(
    {str(row['visi']) for row in result.current_rows},
    {obs['visi'] for obs in observations}
    )
assert_true(all(row['date'] == '0101') for row in result.current_rows)
assert_equal(
    {row['id'] for row in result.current_rows},
    set(range(1, len(observations) + 1))
    )

## Retrieving Data

- Retrieve the `id` attribute of all rows where the `conds` attribute of the `weather` table is equal to `condition` (e.g., "Clear", "Partly Cloudy", etc.).

In [ ]:
def retrieve_data(session, condition):
    '''
    Retrieves the "id" attribute of all rows
    where the "conds" attribute of the "weather" table
    is equal to "condition"
    
    Parameters
    ----------
    session: A cassandra.cluster.Session instance.
    condition: A string, e.g., "Clear", "Partly Cloudy", "Overcast", etc.
    
    Returns
    -------
    A cassandra.cluster.ResultSet instance.

    '''
    
    # YOUR CODE HERE
    
    return result

In [ ]:
retrieve_result = retrieve_data(session, 'Clear')
print(retrieve_result.response_future)

In [ ]:
assert_is_instance(result, cassandra.cluster.ResultSet)

conds = {obs['conds'] for obs in observations}

for cond in conds:
    r = [row['id'] for row in retrieve_data(session, cond).current_rows]
    n = [idx + 1 for idx, obs in enumerate(observations) if obs['conds'] == cond]
    assert_equal(len(r), len(n))
    assert_equal(set(r), set(n))

## Modifying Data

- Change the `visi` value to 1.0 for all rows where the `conds` attribute is equal to `"Clear"`.

In [ ]:
def modify(session):
    '''
    Changes "visi" to 1.0 if "conds" is equal to "Clear".
    
    Parameters
    ----------
    session: A cassandra.cluster.Session instance.
    
    Returns
    -------
    A cassandra.cluster.ResultSet instance.
    '''
    
    # YOUR CODE HERE
    
    return result

In [ ]:
modify_result = modify(session)
print(modify_result.response_future)

In [ ]:
result = session.execute('SELECT * FROM weather;')

assert_equal(
    len([row for row in result.current_rows if row['conds'] == 'Clear']),
    len([obs for obs in observations if obs['conds'] == 'Clear'])
    )

conds = {obs['conds'] for obs in observations}

for cond in conds:
    if cond != 'Clear' and cond != 'Cloudy':
        r = [str(row['visi']) for row in result.current_rows if row['conds'] == cond]
        n = [obs['visi'] for obs in observations if obs['conds'] == cond]
        assert_equal(len(r), len(n))
        assert_equal(set(r), set(n))

## Deleting Data

- Delete all rows where the `conds` attribute is equal to `"Clear"`.

In [ ]:
def delete(session):
    '''
    Delete all rows where "conds" is "Clear".
    
    Parameters
    ----------
    session: A cassandra.cluster.Session instance.
    
    Returns
    -------
    A cassandra.cluster.ResultSet instance.
    '''
    
    # YOUR CODE HERE
    
    return result

In [ ]:
delete_result = delete(session)
print(delete_result.response_future)

In [ ]:
result = session.execute('SELECT * FROM weather;')

assert_equal(len([row for row in result.current_rows if row['conds'] == 'Clear']), 0)

observations = [obs for obs in observations if obs['conds'] != 'Clear']

for cond in conds:
    r = [str(row['visi']) for row in result.current_rows if row['conds'] == cond]
    n = [obs['visi'] for obs in observations if obs['conds'] == cond]
    assert_equal(len(r), len(n))
    assert_equal(set(r), set(n))